# Combine XRD-CT reconstructions in a virtual .h5 file

In [ ]:
import os
import h5py as h5
#To import DanMAX from the folder above:
import sys
sys.path.append('../')
import DanMAX as DM

In [ ]:
scans = []

for i,scan in enumerate(scans):
    fname = DM.findScan(scan).replace('/raw/','/process/xrd_ct/').replace('.h5','_recon.h5')
    dset = '/reconstructed/gridrec'
    with h5.File(fname,'r') as f:
        if i<1:
            layout = h5.VirtualLayout(shape=(len(scans), *f[dset].shape), dtype=f[dset].dtype)
            if 'q' in f.keys():
                Q = True
                x = f['q'][:]
            else:
                Q = False
                x = f['2th'][:]
            if 'micrometer_per_px' in f.keys():
                um_per_px = f['micrometer_per_px'][()]
            else:
                um_per_px = None
        # create virtual source
        vsource = h5.VirtualSource(f[dset])
    # add virtual source to virtual layout
    layout[i] = vsource

# Add virtual dataset to output file
dst = os.path.join(dir,'virtual.h5')
with h5.File(dst, 'w', libver='latest') as f:
    f.create_virtual_dataset(dset, layout, fillvalue=-5)
    if Q:
        f.create_dataset('q',data=x)
    else:
        f.create_dataset('2th',data=x)
    if not um_per_px is None:
        f.create_dataset('micrometer_per_px',data=um_per_px)
